In [29]:
import cv2
import numpy as np
import os

# Function to apply Gamma Correction
def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

# Function for White Balancing
def apply_white_balance(image):
    result = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    avg_a = np.average(result[:, :, 1])
    avg_b = np.average(result[:, :, 2])
    result[:, :, 1] = result[:, :, 1] - ((avg_a - 128) * (result[:, :, 0] / 255.0) * 1.1)
    result[:, :, 2] = result[:, :, 2] - ((avg_b - 128) * (result[:, :, 0] / 255.0) * 1.1)
    return cv2.cvtColor(result, cv2.COLOR_LAB2BGR)

# Function to apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
def apply_clahe(image, clip_limit=2.0, tile_grid_size=(8, 8)):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    lab[:, :, 0] = clahe.apply(lab[:, :, 0])
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

# Function for Color Restoration (focuses on enhancing red tones)
def restore_red_tones(image, red_gain=1.05):
    (B, G, R) = cv2.split(image)
    R = cv2.convertScaleAbs(R, alpha=red_gain, beta=0)
    return cv2.merge([B, G, R])

# Function to adjust brightness
def adjust_brightness(image, brightness_factor=1.03):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hsv[:, :, 2] = cv2.convertScaleAbs(hsv[:, :, 2], alpha=brightness_factor)
    return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

# Function to fine-tune contrast and brightness
def adjust_contrast_brightness(image, contrast_factor=1.05, brightness_factor=5):  # Adjusted to lower values
    return cv2.convertScaleAbs(image, alpha=contrast_factor, beta=brightness_factor)

# Function to apply Gamma Correction to B, G, and R channels
def apply_gamma_correction(image, gamma_blue=1.6, gamma_green=1.4, gamma_red=0.95):
    (B, G, R) = cv2.split(image)
    B = adjust_gamma(B, gamma_blue)
    G = adjust_gamma(G, gamma_green)
    R = adjust_gamma(R, gamma_red)
    return cv2.merge([B, G, R])

# Full pipeline that applies all transformations
def enhance_underwater_image(image):
    # Step 1: Apply White Balancing to correct color imbalance
    white_balanced = apply_white_balance(image)
    
    # Step 2: Apply Gamma Correction to reduce blue/green dominance
    gamma_corrected = apply_gamma_correction(white_balanced, gamma_blue=1.6, gamma_green=1.4, gamma_red=0.95)
    
    # Step 3: Apply CLAHE to enhance local contrast (with reduced clip limit)
    clahe_image = apply_clahe(gamma_corrected, clip_limit=1.5)
    
    # Step 4: Restore Red Tones to compensate for underwater absorption
    red_restored_image = restore_red_tones(clahe_image, red_gain=1.05)
    
    # Step 5: Adjust Brightness for natural lighting
    brightened_image = adjust_brightness(red_restored_image, brightness_factor=1.03)
    
    # Step 6: Fine-tune contrast and brightness with adjusted values
    final_image = adjust_contrast_brightness(brightened_image, contrast_factor=0.9, brightness_factor=3)
    
    return final_image

# Function to merge two images side by side
def merge_images_side_by_side(image1, image2):
    if image1.shape[0] != image2.shape[0]:
        height = min(image1.shape[0], image2.shape[0])
        image1 = cv2.resize(image1, (int(image1.shape[1] * height / image1.shape[0]), height))
        image2 = cv2.resize(image2, (int(image2.shape[1] * height / image2.shape[0]), height))
    merged_image = np.hstack((image1, image2))
    return merged_image

# Function to process images and merge the original and enhanced images side by side
def process_images(original_folder, preprocessed_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    original_images = os.listdir(original_folder)
    
    for image_name in original_images:
        original_image_path = os.path.join(original_folder, image_name)
        preprocessed_image_path = os.path.join(preprocessed_folder, image_name)
        
        if os.path.exists(preprocessed_image_path) and image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            print(f"Processing {image_name}...")
            original_image = cv2.imread(original_image_path)
            preprocessed_image = cv2.imread(preprocessed_image_path)
            
            # Apply the enhancement pipeline to the preprocessed image
            enhanced_image = enhance_underwater_image(preprocessed_image)
            
            # Merge the original and enhanced images side by side
            merged_image = merge_images_side_by_side(original_image, enhanced_image)
            
            # Save the merged image in the output folder
            output_image_path = os.path.join(output_folder, image_name)
            cv2.imwrite(output_image_path, merged_image)
            print(f"Saved merged image: {output_image_path}")

# Example usage with Massimo's folders
original_folder = '/Users/massimoweijtens/Development/DC3/Data'
preprocessed_folder = '/Users/massimoweijtens/Development/DC3/Output'
output_folder = '/Users/massimoweijtens/Development/DC3/Merged'

process_images(original_folder, preprocessed_folder, output_folder)


Processing 7117_Caranx_sexfasciatus_juvenile_f000230.jpg...
Saved merged image: /Users/massimoweijtens/Development/DC3/Merged/7117_Caranx_sexfasciatus_juvenile_f000230.jpg
Processing 7585_F1_f000200.jpg...
Saved merged image: /Users/massimoweijtens/Development/DC3/Merged/7585_F1_f000200.jpg
Processing 7434_F1_f000740.jpg...
Saved merged image: /Users/massimoweijtens/Development/DC3/Merged/7434_F1_f000740.jpg
Processing 9870_Gerres_f000180.jpg...
Saved merged image: /Users/massimoweijtens/Development/DC3/Merged/9870_Gerres_f000180.jpg
Processing 7623_F1_f000410.jpg...
Saved merged image: /Users/massimoweijtens/Development/DC3/Merged/7623_F1_f000410.jpg
Processing 7463_F4_f000010.jpg...
Saved merged image: /Users/massimoweijtens/Development/DC3/Merged/7463_F4_f000010.jpg
Processing 7463_F3_f000020.jpg...
Saved merged image: /Users/massimoweijtens/Development/DC3/Merged/7463_F3_f000020.jpg
Processing 7426_F3_f000170.jpg...
Saved merged image: /Users/massimoweijtens/Development/DC3/Merged/